In [0]:
#TAREA 2, PARTE 1 PUNTO A
import pyspark
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, DateType, IntegerType
from pyspark.sql.functions import year, month, dayofmonth
import pyspark.sql.functions as F

df_schema = StructType() \
            .add(field= "VendorID", data_type=StringType(), nullable= True) \
            .add(field="tpep_pickup_datetime", data_type=DateType(), nullable= True) \
            .add(field="tpep_dropoff_datetime", data_type=DateType(), nullable= True) \
            .add(field="passenger_count", data_type=IntegerType(), nullable= True) \
            .add(field="trip_distance", data_type=DoubleType(), nullable= True) \
            .add(field="RatecodeID", data_type=IntegerType(), nullable= True) \
            .add(field="store_and_fwd_flag", data_type=StringType(), nullable= True) \
            .add(field="PULocationID", data_type=IntegerType(), nullable= True) \
            .add(field="DOLocationID", data_type=IntegerType(), nullable= True) \
            .add(field="payment_type", data_type=IntegerType(), nullable= True) \
            .add(field="fare_amount", data_type=DoubleType(), nullable= True) \
            .add(field="extra", data_type=DoubleType(), nullable= True) \
            .add(field="mta_tax", data_type=DoubleType(), nullable= True) \
            .add(field="tip_amount", data_type=DoubleType(), nullable= True) \
            .add(field="tolls_amount", data_type=DoubleType(), nullable= True) \
            .add(field="improvement_surcharge", data_type=DoubleType(), nullable= True) \
            .add(field="total_amount", data_type=DoubleType(), nullable= True) \
            .add(field="congestion_surcharge", data_type=DoubleType(), nullable= True)

            
df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(df_schema) \
      .load("dbfs:/FileStore/tables/yellow_tripdata_2019_01.csv")
	  
df_with_schema_col = df_with_schema.withColumn("year", year(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("month", month(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("day", dayofmonth(df_with_schema.tpep_pickup_datetime)) 
			  
#display(df_with_schema_col)
#display(df_with_schema_col.groupBy('PULocationID').count())

#pu_location_id = df_with_schema_col.select("PULocationID").collect()






In [0]:

#PARTE 1 PUNTO B
file_location = "/FileStore/tables/nyu_2451_36743_geojson.json"
data = spark.read.json(file_location)

location_id = data.select("features.properties.locationid")
segment_location_id = location_id.select("locationid").collect()
lista_id = segment_location_id[0][0]

name_location = data.select("features.properties.zone")
segment_name_location = name_location.select("zone").collect()
lista_name = segment_name_location[0][0]

#df_name_id = sqlContext.createDataFrame(zip(lista_id, lista_name), schema=['location_id', 'zone'])

id_name_tuple = [(lista_id[i],lista_name[i]) for i in range(len(lista_name))] #esta lista de tuplas me funciona como dicionario id--nombre zona

def name_zone(value_1, lista = id_name_tuple):
  for i,j in lista:
    if(i==value_1):
      return j
  return 'Locacion NO registrada'

name_zone_UDF = F.udf(lambda x:name_zone(x))
  
final_df = df_with_schema_col.withColumn('PULocationZone',name_zone_UDF(df_with_schema_col.PULocationID))\
                .withColumn('DOLocationZone',name_zone_UDF(df_with_schema_col.DOLocationID))
#display(final_df)




In [0]:
import numpy as np
pu_location_list = [row.PULocationID for row in df_with_schema_col.select('PULocationID').collect()]
#pu_tuple = [(pu_location_list_id[i],pu_location_list_name) for i in range(len(pu_location_list_name))]

#do_location_list = [row.DOLocationID for row in df_with_schema_col.select('DOLocationID').collect()]
matriz_OD_zeros = np.zeros([len(lista_id)+3,len(lista_name)+3])


In [0]:
for i in range(len(pu_location_list)):
  matriz_OD_zeros[pu_location_list[i]][do_location_list[i]] +=1 



In [0]:
import seaborn as sns
import pandas as pd

data = pd.DataFrame(matriz_OD_zeros)
#display(data)

In [0]:
#Punto 2D
frecuent_places = final_df.groupBy('DOLocationID').count().orderBy('count',ascending=False)
frecuent_places_names = frecuent_places.withColumn('NameLocation',name_zone_UDF(frecuent_places.DOLocationID))
#display(frecuent_places_names)
